In [1]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
import numpy as np
import h5py

In [2]:

def explained_variance_score(y_true,y_pred):
    error = y_true - y_pred
    var_error = K.sum(K.square(error - K.mean(error)))
    var_true = K.sum(K.square(y_true - K.mean(y_pred)))
    return (1 - (var_error/var_true))


def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

def autoencoder(input_shape):
    model = Sequential(name="Autoencoder")
    # Encoder
    model.add(layers.InputLayer(input_shape= input_shape)) # 240x1x1
    model.add(layers.Conv2D(16, kernel_size=(6, 1),strides=(6,1),activation='tanh',padding='valid',name="conv_1")) # 40x1x16   
    model.add(layers.Conv2D(32,  kernel_size=(3,1),strides=(2,1),activation='tanh',padding='same',name="conv_2")) # 20x1x32
    model.add(layers.Conv2D(64, kernel_size=(3,1),strides=(2,1),activation='tanh',padding='same',name="conv_3")) # 10x1x64 
    model.add(layers.Conv2D(128, kernel_size=(3,1),strides=(2,1),activation='tanh',padding='same',name="conv_4")) # 5x1x128    
    model.add(layers.Flatten(name="flatten_1")) # 640
    # Coding
    model.add(layers.Dense(300,activation='tanh',name="dense_1")) # 300
    model.add(layers.Dense(640,activation='tanh',name="dense_2")) # 640
    # Decoder
    model.add(layers.Reshape((5,1,128),name="reshape_1")) # 5x1x128
    model.add(layers.Conv2DTranspose(64, kernel_size=(3, 1),strides=(2,1),activation='tanh',padding='same',name="deconv_1")) # 10x1x64
    model.add(layers.Conv2DTranspose(32,kernel_size=(3,1), strides=(2, 1), activation='tanh',padding='same',name="deconv_2")) # 20x1x32
    model.add(layers.Conv2DTranspose(16,kernel_size=(3,1), strides=(2, 1), activation='tanh',padding='same',name="deconv_3")) # 40x1x16  
    model.add(layers.Conv2DTranspose(1,kernel_size=(6,1), strides=(6, 1), activation='tanh',padding='valid',name="deconv_4")) # 240x1x1  
    return model



## Read data (anonymized sampling)

In [3]:
#with h5py.File('data_set_5_conv/data_set_5.hdf5', 'a') as data:
#    X_train = data["Data_train/X_train"][:]
#    X_test = data["Data_test/X_test"][:]
    
#X_train = np.reshape(X_train, (len(X_train), 240,1, 1))
#X_test = np.reshape(X_test, (len(X_test), 240, 1,1))

In [ ]:
X_train=np.load("X_train.npy")
X_test=np.load("X_test.npy")

## Training y testing sets

In [4]:
print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)

X_train :  (340964, 240, 1, 1)
X_test :  (137782, 240, 1, 1)


## Architecture of autoencoder

In [17]:
input_shape = X_train.shape[1:] 
model = autoencoder(input_shape)
model.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 40, 1, 16)         112       
_________________________________________________________________
conv_2 (Conv2D)              (None, 20, 1, 32)         1568      
_________________________________________________________________
conv_3 (Conv2D)              (None, 10, 1, 64)         6208      
_________________________________________________________________
conv_4 (Conv2D)              (None, 5, 1, 128)         24704     
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               192300    
_________________________________________________________________
dense_2 (Dense)              (None, 640)               

## Training of neural network

In [18]:
# Configuration
nb_epoch = 100 #ORiginally is minimally enough with 500
batch_size = 32
model.compile(optimizer='sgd', loss='mse',metrics=[r2,explained_variance_score])
model.fit(X_train,X_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(X_test,X_test),
          callbacks=[TensorBoard(log_dir='./data_set_55_conv/logs/')])
model.save("./data_set_55_conv/autoencoder.h5")

Epoch 1/100
1066/1066 [==============================] - 5s 4ms/step - loss: 0.0535 - r2: -0.6156 - explained_variance_score: 0.1929 - val_loss: 0.0332 - val_r2: -0.0465 - val_explained_variance_score: 0.0321
Epoch 2/100
1066/1066 [==============================] - 3s 3ms/step - loss: 0.0334 - r2: 7.7726e-04 - explained_variance_score: 0.0365 - val_loss: 0.0331 - val_r2: -0.0457 - val_explained_variance_score: 0.0314
Epoch 3/100
1066/1066 [==============================] - 3s 3ms/step - loss: 0.0333 - r2: 0.0032 - explained_variance_score: 0.0411 - val_loss: 0.0329 - val_r2: -0.0394 - val_explained_variance_score: 0.0280
Epoch 4/100
1066/1066 [==============================] - 3s 3ms/step - loss: 0.0327 - r2: 0.0109 - explained_variance_score: 0.0487 - val_loss: 0.0329 - val_r2: -0.0391 - val_explained_variance_score: 0.0341
Epoch 5/100
1066/1066 [==============================] - 3s 3ms/step - loss: 0.0330 - r2: 0.0151 - explained_variance_score: 0.0513 - val_loss: 0.0327 - val_r2: -0

Epoch 40/100
1066/1066 [==============================] - 3s 3ms/step - loss: 8.3154e-04 - r2: 0.9750 - explained_variance_score: 0.9752 - val_loss: 7.3486e-04 - val_r2: 0.9768 - val_explained_variance_score: 0.9769
Epoch 41/100
1066/1066 [==============================] - 3s 3ms/step - loss: 8.0852e-04 - r2: 0.9756 - explained_variance_score: 0.9758 - val_loss: 7.2500e-04 - val_r2: 0.9771 - val_explained_variance_score: 0.9772
Epoch 42/100
1066/1066 [==============================] - 3s 3ms/step - loss: 8.0036e-04 - r2: 0.9761 - explained_variance_score: 0.9764 - val_loss: 7.1639e-04 - val_r2: 0.9774 - val_explained_variance_score: 0.9775
Epoch 43/100
1066/1066 [==============================] - 3s 3ms/step - loss: 7.9320e-04 - r2: 0.9762 - explained_variance_score: 0.9764 - val_loss: 7.0685e-04 - val_r2: 0.9777 - val_explained_variance_score: 0.9778
Epoch 44/100
1066/1066 [==============================] - 3s 3ms/step - loss: 7.9399e-04 - r2: 0.9761 - explained_variance_score: 0.9763

Epoch 78/100
1066/1066 [==============================] - 3s 3ms/step - loss: 6.1980e-04 - r2: 0.9814 - explained_variance_score: 0.9815 - val_loss: 5.4915e-04 - val_r2: 0.9827 - val_explained_variance_score: 0.9828
Epoch 79/100
1066/1066 [==============================] - 3s 3ms/step - loss: 6.1516e-04 - r2: 0.9816 - explained_variance_score: 0.9818 - val_loss: 5.4476e-04 - val_r2: 0.9828 - val_explained_variance_score: 0.9829
Epoch 80/100
1066/1066 [==============================] - 3s 3ms/step - loss: 6.1580e-04 - r2: 0.9816 - explained_variance_score: 0.9818 - val_loss: 5.4380e-04 - val_r2: 0.9829 - val_explained_variance_score: 0.9829
Epoch 81/100
1066/1066 [==============================] - 3s 3ms/step - loss: 6.0880e-04 - r2: 0.9816 - explained_variance_score: 0.9817 - val_loss: 5.4277e-04 - val_r2: 0.9829 - val_explained_variance_score: 0.9830
Epoch 82/100
1066/1066 [==============================] - 3s 3ms/step - loss: 6.0639e-04 - r2: 0.9819 - explained_variance_score: 0.9820